# КМ-01 | Романецький Микита | PNN

## Завдання
Розробити програмне забезпечення для реалізації мережі PNN, що призначена для апроксимації функції: $у = х_1 + х_2$  
Передбачити режими навчання та розпізнавання.


In [1]:
import numpy as np
import pandas as pd
from typing import List, Dict, Optional

In [2]:
def train_pnn(X_train: np.ndarray,
              Y_train: List[str],
              delta: float = 1.0) -> Dict[str, float]:
    """
    Фунція навчить імовірнісну нейронну мережу (PNN) за навчальними даними.

    Аргументи:
        - X_train (numpy.ndarray):  Характеристики навчальних даних.
        - Y_train (numpy.ndarray):  Відповідні мітки класів.
        - delta (float):            Гіперпараметр для керування чутливістю до точок даних.

    Повертає значення:
        dict:                       Словник, що містить ймовірності класів.
    """
    class_probs = {}  #  Ініціалізувати порожній словник для зберігання ймовірностей класів
    
    #  У циклі перебираємо навчальні дані та мітки
    for x, class_label in zip(X_train, Y_train):
        if class_label not in class_probs:  #  Якщо мітка класу відсутня у словнику, ініціалізуйте її значенням 0.0
            class_probs[class_label] = 0.0
        """Обчислюємо ймовірності класів за формулою ядра Гауса. Ядро Гауса вимірює схожість між точками даних.
        Воно базується на евклідовій відстані між X_train та х, зваженій на дельту"""
        class_probs[class_label] += np.exp(-np.sum((X_train - x) ** 2, axis=1) / (2 * delta ** 2))

    return class_probs


def predict_pnn(class_probs: Dict[str, float],
                X_predict: np.ndarray,
                X_train: np.ndarray,
                delta: float = 1.0) -> Optional[str]:
    """
    Функція передбачає клас для заданої точки даних за допомогою навченого PNN.

    Аргументи:
        - class_probs (dict):             Словник ймовірностей класів, отриманий у результаті навчання.  
        - X_predict (numpy.ndarray):      Точка даних для розпізнавання.  
        - X_train (numpy.ndarray):        Характеристики навчальних даних.
        - delta (float):                  Гіперпараметр для керування чутливістю до точок даних.  

    Повертається:
        str:                              Розпізнана мітка класу. (Може повернути None)  
    """
    max_prob = 0  #  Ініціалізувуємо максимальну ймовірність рівну 0
    predicted_class = None  #  Ініціалізувуємо розпізнаний клас значенням None

    for class_label, prob in class_probs.items():    
        """У циклі обчислюємо схожість між X_train та X_predict за формулою ядра Гауса
        Ядро Гауса базується на евклідовій відстані між X_train та X_predict, зваженій на дельту."""
        similarity = np.exp(-np.sum((X_train - X_predict) ** 2, axis=1) / (2 * delta ** 2))
        class_probs[class_label] += similarity

        if np.max(prob) > max_prob:  #  Порівняєм максимальне значення імовірності
            max_prob = np.max(prob)
            predicted_class = class_label

    return predicted_class


#### Визначаємо навчальні дані

In [3]:
X_train = np.array([[100, 300], [200, 100], [5.0, 1.0], [300, 200], [8.0, 1.0], [6.0, 6.0]])
Y_train = np.array(['A', 'A', 'B', 'A', 'B', 'B'])

#### Налаштувуємо гіперпараметр для бажаної точності
Чим менше дельта, тим більше модель приділяє увагу невеликим відмінностям між даними

In [4]:
delta = 250.0

#### Тренуємо модель на навчальних даних

In [5]:
class_probs = train_pnn(X_train, Y_train, delta)

#### Визначаємо дані, які потрібно розпізнати

In [6]:
X_predict = np.array([3.0, 1.2])

#### Розпізнаємо клас введених даних

In [7]:
predicted_class = predict_pnn(class_probs, X_predict, X_train, delta)

print(f'Data {X_predict.tolist()} | Predicted class: {predicted_class}')

Data [3.0, 1.2] | Predicted class: B


#### Тестування програми


In [8]:
class_probs = train_pnn(X_train, Y_train, delta)
x_t = [[150, 350], [250, 150], [5.5, 1.5], [350, 250], [8.5, 1.5], [6.5, 6.5], [150, 250], [3.5, 4.5], [2.5, 3.5]]

results_df = pd.DataFrame(columns=["Data", "Predicted class"])

for x in x_t:
    x = np.array(x)
    pred_cl = predict_pnn(class_probs, x, X_train, delta)
    results_df = pd.concat([results_df, pd.DataFrame({"Data": [x.tolist()], "Predicted class": [pred_cl]})],
                           ignore_index=True)

print(results_df)

         Data Predicted class
0  [150, 350]               B
1  [250, 150]               A
2  [5.5, 1.5]               B
3  [350, 250]               A
4  [8.5, 1.5]               A
5  [6.5, 6.5]               B
6  [150, 250]               A
7  [3.5, 4.5]               B
8  [2.5, 3.5]               B


## Висновки
Як бачимо, розпізнавання класів працює добре, із 9 варіантів модель помилилась у двох:  
- [150, 350] predicted class B  
- [8.5, 1.5] predicted class A  

**Примітка: трицифрові числа класс А, двоцифрові класс B**

Тобто точність моделі на даний момент дорівнює $7/9$  

Але варто зауважити, що це не є реальною точністю, оскільки варто зробити навчальну та тестувальну вибірку більшими,   
а гіперпараметр дельта налаштувати для цієї задачі (Можливо перебрати кроссвалідацією)